In [1]:
import numpy as np
import cvxpy as cvx
from qcqp import *
import scipy

In [2]:
# n, m = 10, 15
n, m = 50, 80
np.random.seed(1)

A = np.random.randn(m, n)
b = np.random.randn(m, 1)

x = cvx.Variable(n)
# obj = cvx.sum_squares(A*x - b)
# cons = [cvx.square(x) == 1]
# prob = cvx.Problem(cvx.Minimize(obj), cons)
sni_f0 = (scipy.sparse.csr_matrix(A.T.dot(A)), scipy.sparse.csc_matrix(-2*A.T.dot(b)), b.T.dot(b))
# sni_fs = [(scipy.sparse.csr_matrix(np.eye(n)), scipy.sparse.csc_matrix((n, 1)), -1, "==")]
sni_fs = []
Qi = np.zeros((n, n))
for i  in range(n):
    Qi[i,i] = 1
    sni_fs += [(scipy.sparse.csr_matrix(Qi), scipy.sparse.csc_matrix((n, 1)), -1, "==")]
    Qi[i,i] = 0
prob = QCQPProblem(sni_f0, sni_fs, x)
qcqp = QCQP(prob)


# sample from the semidefinite relaxation
qcqp.suggest(SDR)
print("SDR lower bound: %.4f" % qcqp.sdr_bound)
f_ipopt, v_ipopt = qcqp.improve(COORD_DESCENT)
f_ipopt, v_ipopt = qcqp.improve(IPOPT, verbose=False)
x_ipopt = x.value
print("Coordinate descent + ipopt: objective %.4f, violation %.4f" % (f_ipopt, v_ipopt))

qcqp.suggest(SDR)
print("SDR lower bound: %.4f" % qcqp.sdr_bound)
f_ipopt, v_ipopt = qcqp.improve(IPOPT, verbose=False)
x_ipopt = x.value
print("Ipopt: objective %.4f, violation %.4f" % (f_ipopt, v_ipopt))

# sample from the semidefinite relaxation
qcqp.suggest(SDR)
print("SDR lower bound: %.4f" % qcqp.sdr_bound)
f_admm, v_admm = qcqp.improve(COORD_DESCENT)
f_admm, v_admm = qcqp.improve(ADMM, phase1=False)
x_admm = x.value
print("Coordinate descent + nonconvex ADMM: objective %.4f, violation %.4f" % (f_admm, v_admm))

qcqp.suggest(SDR,) #verbose=True)
# print("SDR lower bound: %.4f" % qcqp.sdr_bound)
f_cd, v_cd = qcqp.improve(COORD_DESCENT)
x_cd = x.value
print("Coordinate descent: objective %.4f, violation %.4f" % (f_cd, v_cd))
# print(f"{x_cd=}")

# SDR solution is cached and not solved again
qcqp.suggest(SDR)
# print("SDR lower bound: %.4f" % qcqp.sdr_bound)
f_dccp, v_dccp = qcqp.improve(DCCP)
print("Penalty CCP: objective %.4f, violation %.4f" % (f_dccp, v_dccp))
f_dccp, v_dccp = qcqp.improve(COORD_DESCENT, phase1=False)
x_dccp = x.value
print("Penalty CCP + coordinate descent: objective %.4f, violation %.4f" % (f_dccp, v_dccp))
# print(f"{x_dccp=}")

SDR lower bound: 518.0990
Coordinate descent + ipopt: objective 1164.7613, violation 0.0000
SDR lower bound: 518.0990
Ipopt: objective 1391.2021, violation 0.0000
SDR lower bound: 518.0990
Coordinate descent + nonconvex ADMM: objective 1804.1780, violation 0.0001
Coordinate descent: objective 1384.8026, violation 0.0071
Penalty CCP: objective 1062.6067, violation 0.0000
Penalty CCP + coordinate descent: objective 995.9132, violation 0.0000
